In [1]:
import nltk
import random
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Download necessary NLTK resources
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# Load intents from JSON file
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Initialize stemmer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """Split sentence into array of words/tokens."""
    return nltk.word_tokenize(sentence)

def stem(word):
    """Find the root form of the word."""
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """Return bag of words array."""
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag

In [4]:
# Prepare training data
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [5]:
# Create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:
# Define the enhanced neural network
class EnhancedNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_hidden_layers=3, dropout_rate=0.3):
        super(EnhancedNeuralNet, self).__init__()

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_size)

        # Hidden layers
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden_layers)]
        )

        # Batch normalization for hidden layers
        self.batch_norms = nn.ModuleList(
            [nn.BatchNorm1d(hidden_size) for _ in range(num_hidden_layers)]
        )

        # Activation function
        self.relu = nn.ReLU()

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)

        # Output layer
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Input layer
        out = self.input_layer(x)
        out = self.relu(out)

        # Hidden layers with BatchNorm, ReLU, and Dropout
        for hidden_layer, batch_norm in zip(self.hidden_layers, self.batch_norms):
            out = hidden_layer(out)
            out = batch_norm(out)
            out = self.relu(out)
            out = self.dropout(out)

        # Output layer
        out = self.output_layer(out)
        return out


In [7]:
# Hyperparameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 128  # Increased hidden size for more capacity
output_size = len(tags)
num_hidden_layers = 4
dropout_rate = 0.3

In [8]:
# Dataset class
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedNeuralNet(input_size, hidden_size, output_size, num_hidden_layers, dropout_rate).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0737
Epoch [200/1000], Loss: 0.2496
Epoch [300/1000], Loss: 0.9444
Epoch [400/1000], Loss: 0.0200
Epoch [500/1000], Loss: 0.0038
Epoch [600/1000], Loss: 0.1753
Epoch [700/1000], Loss: 0.1350
Epoch [800/1000], Loss: 0.0688
Epoch [900/1000], Loss: 0.0133
Epoch [1000/1000], Loss: 0.0236
Final loss: 0.0236


In [10]:
# Save the model
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "chatbot_model.pth"
torch.save(data, FILE)
print(f'Training complete. Model saved to {FILE}')

Training complete. Model saved to chatbot_model.pth


In [11]:
# Chatbot functionality
class ChatBot:
    def __init__(self):
        self.model = model
        self.model.eval()
        self.all_words = all_words
        self.tags = tags

    def get_response(self, user_input):
        """Get a response based on user input."""
        sentence = tokenize(user_input)
        X = bag_of_words(sentence, self.all_words)
        X = torch.from_numpy(X).unsqueeze(0).to(device)  # Add batch dimension

        with torch.no_grad():
            output = self.model(X)
            _, predicted = torch.max(output, dim=1)
            tag = self.tags[predicted.item()]

        # Find the corresponding intent and return a response
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])

        return "I'm sorry, I don't understand."

In [12]:
# Chat with the bot
chatbot = ChatBot()
print("Let's chat! (type 'quit' to exit)")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot.get_response(user_input)
    print(f"Bot: {response}")

Let's chat! (type 'quit' to exit)
You: quit


In [14]:
def validate_model(test_data, model, all_words, tags, device):
    """
    Validate the chatbot model on a test dataset.

    Parameters:
        test_data (list): A list of tuples where each tuple contains a user input and its expected tag.
                          Example: [("How can I apply?", "admissions"), ...]
        model (EnhancedNeuralNet): The trained chatbot model.
        all_words (list): The vocabulary used for training.
        tags (list): The list of intent tags.
        device (torch.device): The device on which the model is running (CPU or CUDA).

    Returns:
        float: Accuracy of the model on the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = len(test_data)

    with torch.no_grad():  # Turn off gradients for evaluation
        for user_input, expected_tag in test_data:
            # Preprocess user input
            tokenized_input = tokenize(user_input)
            bag = bag_of_words(tokenized_input, all_words)
            bag_tensor = torch.from_numpy(bag).unsqueeze(0).to(device)  # Add batch dimension

            # Get the model prediction
            output = model(bag_tensor)
            _, predicted_idx = torch.max(output, dim=1)
            predicted_tag = tags[predicted_idx.item()]

            # Compare with the expected tag
            if predicted_tag == expected_tag:
                correct += 1

    accuracy = (correct / total) * 100
    return accuracy

# Example Test Data (can be expanded or modified)
test_data = [
    # Admissions-related questions
    ("How can I apply for admission?", "admissions"),
    ("What are the admission requirements?", "admissions"),
    ("When is the application deadline?", "admissions"),
    ("Can I apply online?", "admissions"),
    ("Is the admission process open now?", "admissions"),
    ("Where can I find the admission form?", "admissions"),
    ("What documents are needed for admission?", "admissions"),
    ("Do you have rolling admissions?", "admissions"),

    # Courses-related questions
    ("Can I see the list of courses?", "courses"),
    ("What courses are offered?", "courses"),
    ("Do you have engineering programs?", "courses"),
    ("What are the popular courses?", "courses"),
    ("Can you tell me about business programs?", "courses"),
    ("Do you offer data science programs?", "courses"),
    ("Are there any evening courses?", "courses"),
    ("Is there a course in AI?", "courses"),

    # Fees-related questions
    ("How much does the MBA program cost?", "fees"),
    ("What is the tuition fee?", "fees"),
    ("Are there any scholarships available?", "fees"),
    ("What is the fee structure?", "fees"),
    ("Is there any financial aid for students?", "fees"),
    ("Do international students pay more fees?", "fees"),
    ("Can I pay fees in installments?", "fees"),
    ("What is the cost for a bachelor's program?", "fees"),

    # Transportation-related questions
    ("Do you provide transportation facilities?", "transportation"),
    ("Is there a bus service for students?", "transportation"),
    ("What are the transport charges?", "transportation"),
    ("Do you offer transportation on weekends?", "transportation"),
    ("Can I get a bus schedule?", "transportation"),
    ("How far does the campus transportation go?", "transportation"),
    ("Is the transportation service reliable?", "transportation"),

    # Events-related questions
    ("Tell me about campus events.", "events"),
    ("Are there any upcoming cultural events?", "events"),
    ("What events are happening this month?", "events"),
    ("Do you host any workshops?", "events"),
    ("When is the next seminar?", "events"),
    ("What activities are planned for freshmen?", "events"),
    ("Is there a sports fest?", "events"),

    # Placement-related questions
    ("What are the placement statistics?", "placement"),
    ("Do you have a placement cell?", "placement"),
    ("Which companies visit for placements?", "placement"),
    ("What is the placement process?", "placement"),
    ("Can I get placement support as an alumni?", "placement"),
    ("What is the average package for engineering students?", "placement"),
    ("Do startups hire from your campus?", "placement"),

    # Facilities-related questions
    ("Is there a library on campus?", "facilities"),
    ("What facilities are available?", "facilities"),
    ("Is there a sports complex?", "facilities"),
    ("Can you tell me about hostel facilities?", "facilities"),
    ("Do you have a gym on campus?", "facilities"),
    ("Are there labs for research students?", "facilities"),
    ("What amenities are there for international students?", "facilities"),

    # Library-related questions
    ("What are the library timings?", "library"),
    ("Can I borrow books from the library?", "library"),
    ("Do you have an online library?", "library"),
    ("Are there study rooms in the library?", "library"),
    ("Does the library provide access to research papers?", "library"),
    ("Can I use the library as an external member?", "library"),

    # Scholarships-related questions
    ("Are there any scholarships available?", "scholarships"),
    ("How can I apply for scholarships?", "scholarships"),
    ("Do you offer merit-based scholarships?", "scholarships"),
    ("What are the eligibility criteria for scholarships?", "scholarships"),
    ("Are there scholarships for international students?", "scholarships"),
    ("Can I apply for multiple scholarships?", "scholarships"),
    ("Is financial aid available for low-income families?", "scholarships"),

    # Hostel-related questions
    ("What are the hostel charges?", "hostel"),
    ("Do you have separate hostels for boys and girls?", "hostel"),
    ("What amenities are provided in the hostel?", "hostel"),
    ("Is there 24/7 security in the hostel?", "hostel"),
    ("Can I get a single room in the hostel?", "hostel"),
    ("Do hostels have Wi-Fi?", "hostel"),
    ("Are meals included in the hostel fee?", "hostel"),
]


# Validate the model accuracy
accuracy = validate_model(test_data, model, all_words, tags, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 86.11%
